In [9]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

# Create smaller mtl, sky and truth files

In [11]:
initial_mtl_file = "targets/subset_dr8_mtl_dark_gray_NGC.fits"
if not os.path.exists(initial_mtl_file):
    mtl_file = "targets/dr8_mtl_dark_gray_northern_cap.fits"
    mtl_data = Table.read(mtl_file)
    subset_ii = (mtl_data['RA']>145) & (mtl_data['RA']<205)
    subset_ii &= (mtl_data['DEC']>-5) & (mtl_data['DEC']<25)
    mtl_data[subset_ii].write(initial_mtl_file, overwrite=True)

In [12]:
initial_sky_file = "targets/subset_dr8_sky.fits"
if not os.path.exists(initial_sky_file):
    sky_data = Table.read("/project/projectdirs/desi/target/catalogs/dr8/0.31.0/skies/skies-dr8-0.31.0.fits")
    subset_ii = (sky_data['RA']>145) & (sky_data['RA']<205)
    subset_ii &= (sky_data['DEC']>-5) & (sky_data['DEC']<25)
    print('writing sky')
    sky_data[subset_ii].write(initial_sky_file, overwrite=True)
    print('done writing sky')

In [14]:
initial_truth_file = "targets/subset_truth_dr8_mtl_dark_gray_NGC.fits"
if not os.path.exists(initial_truth_file):
    import desitarget.mock.mockmaker as mb
    from desitarget.targetmask import desi_mask, bgs_mask, mws_mask

    targets = Table.read(initial_mtl_file)
    colnames = list(targets.dtype.names)
    print(colnames)
    nobj = len(targets)
    truth = mb.empty_truth_table(nobj=nobj)[0]
    print(truth.keys())

    for k in colnames:
        if k in truth.keys():
            print(k)
            truth[k][:] = targets[k][:]

    nothing = '          '
    truth['TEMPLATESUBTYPE'] = np.repeat(nothing, nobj)

    masks = ['MWS_ANY', 'BGS_ANY', 'STD_FAINT', 'STD_BRIGHT','ELG', 'LRG', 'QSO', ]
    dict_truespectype = {'BGS_ANY':'GALAXY', 'ELG':'GALAXY', 'LRG':'GALAXY', 'QSO':'QSO', 
                    'MWS_ANY':'STAR', 'STD_FAINT':'STAR', 'STD_BRIGHT':'STAR'}
    dict_truetemplatetype = {'BGS_ANY':'BGS', 'ELG':'ELG', 'LRG':'LRG', 'QSO':'QSO', 
                        'MWS_ANY':'STAR', 'STD_FAINT':'STAR', 'STD_BRIGHT':'STAR'}

    for m in masks:
        istype = (targets['DESI_TARGET'] & desi_mask.mask(m))!=0
        print(m, np.count_nonzero(istype))
        truth['TRUESPECTYPE'][istype] = np.repeat(dict_truespectype[m], np.count_nonzero(istype))
        truth['TEMPLATETYPE'][istype] = np.repeat(dict_truetemplatetype[m], np.count_nonzero(istype))
        truth['MOCKID'][istype] = targets['TARGETID'][istype]

    # Check that all targets have been assigned to a class
    iii = truth['MOCKID']==0
    assert np.count_nonzero(iii)==0
    
    print('writing truth')
    truth.write(initial_truth_file, overwrite=True)
    print('done truth')

In [15]:
tiles = Table(desimodel.io.load_tiles())

ii_tiles = tiles['PROGRAM'] != 'BRIGHT'
ii_tiles &= tiles['RA'] > 150 
ii_tiles &= tiles['RA'] < 200
ii_tiles &= tiles['DEC'] > 0
ii_tiles &= tiles['DEC'] < 20

tilefile = 'footprint/subset_tiles.fits'
tiles[ii_tiles].write(tilefile, overwrite='True')
tiles = Table.read(tilefile)

ii_gray = tiles['PROGRAM']=='GRAY'
ii_dark_0 = (tiles['PROGRAM']=='DARK') & (tiles['PASS']==0)
ii_dark_1 = (tiles['PROGRAM']=='DARK') & (tiles['PASS']==1)
ii_dark_2 = (tiles['PROGRAM']=='DARK') & (tiles['PASS']==2)
ii_dark_3 = (tiles['PROGRAM']=='DARK') & (tiles['PASS']==3)

footprint = dict()
footprint['gray'] = tiles[ii_gray]
footprint['dark0'] = tiles[ii_dark_0]
footprint['dark1'] = tiles[ii_dark_1]
footprint['dark2'] = tiles[ii_dark_2]
footprint['dark3'] = tiles[ii_dark_3]

footprint['gray'].write('footprint/subset_gray.fits', overwrite=True)
footprint['dark0'].write('footprint/subset_dark0.fits', overwrite=True)
footprint['dark1'].write('footprint/subset_dark1.fits', overwrite=True)
vstack([footprint['dark2'], footprint['dark3']]).write('footprint/subset_dark2_dark3.fits', overwrite=True)
vstack([footprint['dark1'], footprint['dark2'], footprint['dark3']]).write('footprint/subset_dark1_dark2_dark3.fits', overwrite=True)
vstack([footprint['dark0'], footprint['dark1'], footprint['dark2'], footprint['dark3']]).write('footprint/subset_dark0_dark1_dark2_dark3.fits', overwrite=True)
vstack([footprint['gray'], footprint['dark0'], footprint['dark1'], footprint['dark2'], footprint['dark3']]).write('footprint/subset_gray_dark0_dark1_dark2_dark3.fits', overwrite=True)

In [19]:
pass_names = ['gray', 'dark0', 'dark1', 'dark2_dark3', 'full']
strategy = 'strategy_A'
for i_pass in range(4):
    
    old_pass_name = pass_names[i_pass-1]
    pass_name = pass_names[i_pass]
    new_pass_name = pass_names[i_pass+1]
    
    os.makedirs('{}/fiberassign_{}'.format(strategy, pass_name), exist_ok=True)
    
    footprint_filename = 'footprint/subset_{}.fits'.format(pass_name)
    fiberassign_dir = '{}/fiberassign_{}/'.format(strategy, pass_name)
    mtl_filename = '{}/targets/{}_subset_dr8_mtl_dark_gray_northern_cap.fits'.format(strategy, pass_name)
    new_mtl_filename = '{}/targets/{}_subset_dr8_mtl_dark_gray_northern_cap.fits'.format(strategy, new_pass_name)
    old_zcat_filename = '{}/zcat/{}_zcat.fits'.format(strategy, old_pass_name)
    zcat_filename = '{}/zcat/{}_zcat.fits'.format(strategy, pass_name)
    
    if i_pass == 0:
        shutil.copyfile(initial_mtl_file, mtl_filename)
        
    
    # Run fiberassign
    cmd = 'fiberassign --mtl {} --sky targets/subset_dr8_sky.fits '.format(mtl_filename)
    cmd +=' --footprint {} --outdir {} --overwrite'.format(footprint_filename, fiberassign_dir)
    print(cmd)
    ! $cmd
    
    # Run qa
    cmd = "fba_run_qa --dir {} --footprint {}".format(fiberassign_dir, footprint_filename)
    print(cmd)
    ! $cmd
    
    # Gather fiberassign files
    fba_files = glob.glob(os.path.join(fiberassign_dir,"tile*.fits"))
    
    # Read targets and truth
    targets = Table.read(mtl_filename)
    truth = Table.read(initial_truth_file)
    
    # Compute zcat
    if i_pass==0:
        zcat = desisim.quickcat.quickcat(fba_files, targets, truth, perfect=True)
    else:
        old_zcat = Table.read(old_zcat_filename)
        zcat = desisim.quickcat.quickcat(fba_files, targets, truth, zcat=old_zcat, perfect=True)        
    
    zcat.write(zcat_filename, overwrite=True)
    mtl = desitarget.mtl.make_mtl(targets, zcat)
    mtl.write(new_mtl_filename, overwrite=True)

fiberassign --mtl strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits --sky targets/subset_dr8_sky.fits  --footprint footprint/subset_gray.fits --outdir strategy_A/fiberassign_gray/ --overwrite
INFO: Reading fiber positions from /global/common/software/desi/cori/desiconda/20180709-1.2.6-spec/code/desimodel/master/data/focalplane/fiberpos-all.fits
INFO: Target file strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits has 8353255 rows.  Reading in chunks of 1000000
INFO: Read target file strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits:  11.28 seconds (1 calls)
INFO: Target file targets/subset_dr8_sky.fits has 9094451 rows.  Reading in chunks of 1000000
INFO: Read target file targets/subset_dr8_sky.fits:  13.15 seconds (1 calls)
INFO: Building target tree:  5.94 seconds (1 calls)
INFO: Computing targets available to all tile / locations:  15.69 seconds (1 calls)
INFO: Computing tile / locations available to all objects:  10.92 seconds (1 cal

# Global statistics

In [23]:
def consolidate_favail(fba_files):
    # getting all the targetids of the assigned fibers
    print('reading the individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%20 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FAVAIL', header=True)
        favail.extend(id_favail['TARGETID'])
    return list(set(favail))

def global_efficiency(targets, id_avail, zcat, target_class='QSO'):
    ii_avail = np.in1d(targets['TARGETID'], id_avail)
    targets_avail = targets[ii_avail]


    is_class = (targets_avail['DESI_TARGET'] & desi_mask.mask(target_class))!=0
    targets_avail_class = targets_avail[is_class]
    n_avail = len(targets_avail_class)

    is_class_from_zcat = np.in1d(targets_avail_class['TARGETID'], zcat['TARGETID'])
    targets_zcat_class = targets_avail_class[is_class_from_zcat]
    n_assigned = len(targets_zcat_class)

    is_class_in_zcat = np.in1d(zcat['TARGETID'], targets_zcat_class['TARGETID'])
    zcat_class = zcat[is_class_in_zcat]

    nobs = dict()
    for i in range(10):
        nobs[i] = np.count_nonzero(zcat_class['NUMOBS']==i)
    nobs[0] = (n_avail - n_assigned)

    print(target_class, n_assigned/n_avail, n_avail, n_assigned, nobs)
    
def tile_efficiency(qa_json_file):
    f = open(qa_json_file)
    qa_dict = json.load(f)
    f.close()
    assign_total = []
    assign_science= []
    assign_sky = []
    assign_std = []
    for k in qa_dict:
        assign_total.append(qa_dict[k]['assign_total'])
        assign_science.append(qa_dict[k]['assign_science'])
        assign_sky.append(qa_dict[k]['assign_sky'])
        assign_std.append(qa_dict[k]['assign_std'])
    assign_total = np.array(assign_total)
    assign_science = np.array(assign_science)
    assign_sky = np.array(assign_sky)
    assign_std = np.array(assign_std)
    n_not_enough_sky = np.count_nonzero(assign_sky<400)
    n_not_enough_std = np.count_nonzero(assign_std<100)
    f_unassigned = (5000 - assign_total)/5000
    print(n_not_enough_sky, n_not_enough_std, np.median(f_unassigned))

In [ ]:
# mkdir fiberassign_full
# cp fiberassign_dark0/*.fits fiberassign_full/
# cp fiberassign_dark1/*.fits fiberassign_full/
# cp fiberassign_dark2_dark3/*.fits fiberassign_full/
# cp fiberassign_gray/*.fits fiberassign_full/
# fba_run_qa --dir fiberassign_full/

In [21]:
fba_files = glob.glob('strategy_A/fiberassign_full/tile-*fits')
favail = consolidate_favail(fba_files)

reading the individual fiberassign files
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620


In [24]:
tile_efficiency('strategy_A/fiberassign_full/qa.json')
global_efficiency(targets, favail, zcat, target_class='QSO')
global_efficiency(targets, favail, zcat, target_class='LRG')
global_efficiency(targets, favail, zcat, target_class='ELG')
global_efficiency(targets, favail, zcat, target_class='LRG_1PASS')
global_efficiency(targets, favail, zcat, target_class='LRG_2PASS')

0 0 0.0
QSO 0.9846025174883907 285761 281361 {0: 4400, 1: 204626, 2: 42243, 3: 24296, 4: 9357, 5: 799, 6: 40, 7: 0, 8: 0, 9: 0}
LRG 0.9359157620916744 473611 443260 {0: 30351, 1: 267029, 2: 127036, 3: 32792, 4: 14654, 5: 1627, 6: 119, 7: 3, 8: 0, 9: 0}
ELG 0.6437995757050582 2551527 1642672 {0: 908855, 1: 1589764, 2: 19517, 3: 23249, 4: 9309, 5: 794, 6: 39, 7: 0, 8: 0, 9: 0}
LRG_1PASS 0.9218001018395501 341714 314992 {0: 26722, 1: 248832, 2: 23523, 3: 26600, 4: 14291, 5: 1624, 6: 119, 7: 3, 8: 0, 9: 0}
LRG_2PASS 0.9724861065831671 131897 128268 {0: 3629, 1: 18197, 2: 103513, 3: 6192, 4: 363, 5: 3, 6: 0, 7: 0, 8: 0, 9: 0}


In [36]:
footprint_names = ['gray_dark0_dark1_dark2_dark3', 'dark0_dark1_dark2_dark3', 'dark1_dark2_dark3', 'dark2_dark3', 'full']
pass_names = ['gray', 'dark0', 'dark1', 'dark2_dark3', 'full']


strategy = 'strategy_B'
for i_pass in range(4):
    
    footprint_name = footprint_names[i_pass]
    old_pass_name = pass_names[i_pass-1]
    pass_name = pass_names[i_pass]
    new_pass_name = pass_names[i_pass+1]
    
    os.makedirs('{}/fiberassign_{}'.format(strategy, pass_name), exist_ok=True)
    
    assign_footprint_filename = 'footprint/subset_{}.fits'.format(footprint_name)
    zcat_footprint_filename = 'footprint/subset_{}.fits'.format(pass_name)
    fiberassign_dir = '{}/fiberassign_{}/'.format(strategy, pass_name)
    mtl_filename = '{}/targets/{}_subset_dr8_mtl_dark_gray_northern_cap.fits'.format(strategy, pass_name)
    new_mtl_filename = '{}/targets/{}_subset_dr8_mtl_dark_gray_northern_cap.fits'.format(strategy, new_pass_name)
    old_zcat_filename = '{}/zcat/{}_zcat.fits'.format(strategy, old_pass_name)
    zcat_filename = '{}/zcat/{}_zcat.fits'.format(strategy, pass_name)
    
    if i_pass == 0:
        shutil.copyfile(initial_mtl_file, mtl_filename)
        
    
    # Run fiberassign
    cmd = 'fiberassign --mtl {} --sky targets/subset_dr8_sky.fits '.format(mtl_filename)
    cmd +=' --footprint {} --outdir {} --overwrite'.format(assign_footprint_filename, fiberassign_dir)
    print(cmd)
    ! $cmd
    
    # Gather fiberassign files
    tile_files = np.sort(glob.glob(os.path.join(fiberassign_dir,"tile*.fits")))
    fba_files = np.sort(glob.glob(os.path.join(fiberassign_dir,"fiberassign*.fits")))

    # remove tilefiles that are not in the list of tiles to build zcat
    footprint = Table.read(zcat_footprint_filename)
    for tile_file, fba_file in zip(tile_files, fba_files):
        fibassign, header = fits.getdata(fba_file, header=True)
        tileid = header['TILEID'] 
        if tileid not in footprint['TILEID']:
            print(tileid, 'not in list', zcat_footprint_filename)
            print('removing {}'.format(fba_file))
            print('removing {}'.format(tile_file))
            os.remove(fba_file)
            os.remove(tile_file)
            
            
    # Run qa
    cmd = "fba_run_qa --dir {} --footprint {}".format(fiberassign_dir, zcat_footprint_filename)
    print(cmd)
    ! $cmd
    
    # Gather fiberassign files
    fba_files = glob.glob(os.path.join(fiberassign_dir,"tile*.fits"))
  
    
    # Read targets and truth
    targets = Table.read(mtl_filename)
    truth = Table.read(initial_truth_file)
    
    # Compute zcat
    if i_pass==0:
        zcat = desisim.quickcat.quickcat(fba_files, targets, truth, perfect=True)
    else:
        old_zcat = Table.read(old_zcat_filename)
        zcat = desisim.quickcat.quickcat(fba_files, targets, truth, zcat=old_zcat, perfect=True)        
    
    zcat.write(zcat_filename, overwrite=True)
    mtl = desitarget.mtl.make_mtl(targets, zcat)
    mtl.write(new_mtl_filename, overwrite=True)

fiberassign --mtl strategy_B/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits --sky targets/subset_dr8_sky.fits  --footprint footprint/subset_gray_dark0_dark1_dark2_dark3.fits --outdir strategy_B/fiberassign_gray/ --overwrite
fba_run_qa --dir strategy_B/fiberassign_gray/ --footprint footprint/subset_gray.fits
INFO: Reading fiber positions from /global/common/software/desi/cori/desiconda/20180709-1.2.6-spec/code/desimodel/master/data/focalplane/fiberpos-all.fits
INFO: Found 125 fiberassign tile files
INFO: Selecting 125 fiberassign tile files
INFO: Processing tile 28367
INFO: Processing tile 28523
INFO: Processing tile 28516
INFO: Processing tile 28460
INFO: Processing tile 28445
INFO: Processing tile 25852
INFO: Processing tile 25839
INFO: Processing tile 25842
INFO: Processing tile 28462
INFO: Processing tile 28485
INFO: Processing tile 25825
INFO: Processing tile 28346
INFO: Processing tile 25841
INFO: Processing tile 28496
INFO: Processing tile 25848
INFO: Processing tile 284

In [32]:
tile_files = glob.glob(os.path.join(fiberassign_dir,"tile*.fits"))


In [35]:
np.sort(tile_files)

array(['strategy_B/fiberassign_gray/tile-025787.fits',
       'strategy_B/fiberassign_gray/tile-025788.fits',
       'strategy_B/fiberassign_gray/tile-025789.fits',
       'strategy_B/fiberassign_gray/tile-025790.fits',
       'strategy_B/fiberassign_gray/tile-025791.fits',
       'strategy_B/fiberassign_gray/tile-025792.fits',
       'strategy_B/fiberassign_gray/tile-025807.fits',
       'strategy_B/fiberassign_gray/tile-025808.fits',
       'strategy_B/fiberassign_gray/tile-025809.fits',
       'strategy_B/fiberassign_gray/tile-025810.fits',
       'strategy_B/fiberassign_gray/tile-025811.fits',
       'strategy_B/fiberassign_gray/tile-025812.fits',
       'strategy_B/fiberassign_gray/tile-025823.fits',
       'strategy_B/fiberassign_gray/tile-025824.fits',
       'strategy_B/fiberassign_gray/tile-025825.fits',
       'strategy_B/fiberassign_gray/tile-025826.fits',
       'strategy_B/fiberassign_gray/tile-025827.fits',
       'strategy_B/fiberassign_gray/tile-025828.fits',
       'st